## Setup

In [116]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.svm import SVR
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error

from scipy.stats import loguniform

In [34]:
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [35]:
housing["median_income"].describe()

count    20640.000000
mean         3.870671
std          1.899822
min          0.499900
25%          2.563400
50%          3.534800
75%          4.743250
max         15.000100
Name: median_income, dtype: float64

In [36]:
# Notice that 75% of the values are below 4.0
# Notice also that the mean=3.1 and std=1.3
pd.DataFrame(np.ceil(housing["median_income"] / 1.5)).describe()

,median_income
count,20640.000000
mean,3.093362
std,1.303707
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,11.000000


In [37]:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5, inplace=True)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,income_cat
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,5.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,5.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,5.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,4.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,3.0


In [38]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# Stratification is done based on the y parameter
for train_index, test_index in sss.split(X=housing, y=housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [39]:
housing_income_cat_ratio = housing["income_cat"].value_counts() / len(housing)
train_income_cat_ratio = strat_train_set["income_cat"].value_counts() / len(strat_train_set)
test_income_cat_ratio = strat_test_set["income_cat"].value_counts() / len(strat_test_set)

housing_df = housing_income_cat_ratio.rename("housing")
train_df = train_income_cat_ratio.rename("train")
test_df = test_income_cat_ratio.rename("test")

combined_df = pd.merge(left=housing_df, right=train_df, how="outer", on="income_cat")
combined_df = pd.merge(left=combined_df, right=test_df, how="outer", on="income_cat")

print(combined_df)

             housing     train      test
income_cat                              
3.0         0.350581  0.350594  0.350533
2.0         0.318847  0.318859  0.318798
4.0         0.176308  0.176296  0.176357
5.0         0.114438  0.114462  0.114341
1.0         0.039826  0.039789  0.039971


In [40]:
for set in (strat_train_set, strat_test_set):
    set.drop("income_cat", axis=1, inplace=True)

In [41]:
X = strat_train_set.drop("median_house_value", axis=1)
X_numerical = X.drop("ocean_proximity", axis=1)
y = strat_train_set["median_house_value"].copy()

len(X) == len(X_numerical) == len(y)

True

In [42]:
numerical_attribs = list(X_numerical)
categorical_attribs = ["ocean_proximity"]

In [45]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.attribute_names].values

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

class MyLabelBinarizer(LabelBinarizer):
    def fit(self, X, y=None):
        return super().fit(X)

    def transform(self, X, y=None):
        return super().transform(X)

    def fit_transform(self, X, y=None):
        return super().fit(X).transform(X)

In [47]:
# steps is a list of (name, transform) tuples
numerical_pipeline = Pipeline(steps=[
    ("selector", DataFrameSelector(numerical_attribs)),
    ("imputer", SimpleImputer(strategy="median")),
    ("attr_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler()),
])

categorical_pipeline = Pipeline(steps=[
    ("selector", DataFrameSelector(categorical_attribs)),
    ("label_binarizer", MyLabelBinarizer()),
])

preprocessing_pipeline = FeatureUnion(transformer_list=[
    ("numerical_pipeline", numerical_pipeline),
    ("categorical_pipeline", categorical_pipeline),
])

In [54]:
X_preprocessed = preprocessing_pipeline.fit_transform(X)
X_preprocessed.shape

(16512, 16)

In [55]:
def display_scores(scores):
    print(f"Scores:\t {scores}")
    print(f"Mean:\t {np.mean(scores)}")
    print(f"Standard Deviation:\t {np.std(scores)}")

## Question 1

Try a support vector regressor (`sklearn.svm.SVR`), with various hyperparameters such as `kernel="linear"` (with various values for the `C` hyperparameter) or `kernel="rbf"` (with various values for the `C` and `gamma` hyperparameters). Don't worry about what these hyperparameters mean for now. How does the best SVR predictor perform?

In [61]:
# param_grid is a list of dictionaries where the keys are parameter names
# and the values are list of parameter settings to try

param_grid = [
    {"kernel": ["linear"], "C": [0.1, 1, 10]},
    {"kernel": ["rbf"], "C": [0.1, 1, 10], "gamma": [0.01, 0.1, 1]},
]

svr = SVR()
grid_search = GridSearchCV(svr, param_grid, scoring="neg_mean_squared_error", cv=5)
grid_search.fit(X_preprocessed, y)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.1, 1, 10], 'kernel': ['linear']},
                         {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error')

In [72]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"{np.sqrt(-mean_score):.2f}\t{params}")

118258.82	{'C': 0.1, 'kernel': 'linear'}
112571.06	{'C': 1, 'kernel': 'linear'}
84649.61	{'C': 10, 'kernel': 'linear'}
118926.92	{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
118906.44	{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
118935.65	{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
118819.34	{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
118643.67	{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
118898.89	{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
117862.26	{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
116181.25	{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
118591.65	{'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [78]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best estimator:\t {grid_search.best_estimator_}")
print(f"Best score:\t {np.sqrt(-grid_search.best_score_)}")

Best parameters: {'C': 10, 'kernel': 'linear'}
Best estimator:	 SVR(C=10, kernel='linear')
Best score:	 84649.6069847477


There are a few things to note here:
- The best estimator found uses a linear kernel
- The best estimator has `'C':10` (this is highest value provided for grid search)
- The best estimator has a score worse than the RandomForestRegressor

## Question 2

Try replacing `GridSearchCV` with `RandomizedSearchCV`.

In [82]:
param_distribs = {
    "svr__kernel": ["linear"],
    "svr__C": loguniform(10, 100000),
}

svr_pipeline = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("svr", SVR()),
])

randomized_search = RandomizedSearchCV(svr_pipeline, param_distribs, n_iter=10, cv=3,
                                       scoring="neg_root_mean_squared_error", random_state=42, verbose=3)
randomized_search.fit(X.iloc[:5000], y.iloc[:5000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END svr__C=314.891164795686, svr__kernel=linear;, score=-73463.527 total time=   0.6s
[CV 2/3] END svr__C=314.891164795686, svr__kernel=linear;, score=-68280.018 total time=   0.6s
[CV 3/3] END svr__C=314.891164795686, svr__kernel=linear;, score=-65332.015 total time=   0.6s
[CV 1/3] END svr__C=63512.21010640709, svr__kernel=linear;, score=-71988.026 total time=   1.4s
[CV 2/3] END svr__C=63512.21010640709, svr__kernel=linear;, score=-66340.143 total time=   1.3s
[CV 3/3] END svr__C=63512.21010640709, svr__kernel=linear;, score=-64331.951 total time=   1.6s
[CV 1/3] END svr__C=8471.801418819978, svr__kernel=linear;, score=-72056.595 total time=   0.7s
[CV 2/3] END svr__C=8471.801418819978, svr__kernel=linear;, score=-66567.772 total time=   0.7s
[CV 3/3] END svr__C=8471.801418819978, svr__kernel=linear;, score=-64348.120 total time=   0.8s
[CV 1/3] END svr__C=2481.0409748678126, svr__kernel=linear;, score=-72223.436 

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              FeatureUnion(transformer_list=[('numerical_pipeline',
                                                                              Pipeline(steps=[('selector',
                                                                                               DataFrameSelector(attribute_names=['longitude',
                                                                                                                                  'latitude',
                                                                                                                                  'housing_median_age',
                                                                                                                                  'total_rooms',
                                                                                                                                  'total_bedrooms',
                                                                                                                                  'population',
                                                                                                                                  'households',
                                                                                                                                  'median_income'])),
                                                                                              ('imputer',
                                                                                               SimpleImputer(strategy='median')),
                                                                                              ('...
                                                                             ('categorical_pipeline',
                                                                              Pipeline(steps=[('selector',
                                                                                               DataFrameSelector(attribute_names=['ocean_proximity'])),
                                                                                              ('label_binarizer',
                                                                                               MyLabelBinarizer())]))])),
                                             ('svr', SVR())]),
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78aeb93def90>,
                                        'svr__kernel': ['linear']},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=3)

In [85]:
print(f"Best parameters: {randomized_search.best_params_}")
print(f"Best estimator:\t {randomized_search.best_estimator_.named_steps["svr"]}")
print(f"Best score:\t {-randomized_search.best_score_}")

Best parameters: {'svr__C': 63512.21010640709, 'svr__kernel': 'linear'}
Best estimator:	 SVR(C=63512.21010640709, kernel='linear')
Best score:	 67553.37339378493


## Question 3

Try adding a transformer in the preparation pipeline to select only the most important attributes.

In [92]:
# SVR does not have feature_importances_ so we need to use RandomForestRegressor
# in order to select the most important features

selector_pipeline = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("feature_selector", SelectFromModel(RandomForestRegressor(random_state=42), threshold=0.05)),
    ("svr", SVR(kernel=randomized_search.best_params_["svr__kernel"],
                C=randomized_search.best_params_["svr__C"])),
])

In [94]:
selector_rmses = -cross_val_score(selector_pipeline, X.iloc[:5000], y.iloc[:5000],
                                  scoring="neg_root_mean_squared_error", cv=3)
selector_rmses

array([75150.25844066, 70580.81387033, 67425.38607789])

## Question 4

Try creating a single pipeline that does the full data preparation plus the final prediction.

In [109]:
class Predictor(BaseEstimator, TransformerMixin):
    def __init__(self, estimator,):
        self.estimator = estimator

    def fit(self, X, y=None):
        estimator_ = clone(self.estimator)
        estimator_.fit(X, y)
        self.estimator = estimator_
        return self

    def transform(self, X, y=None):
        predictions = self.estimator.predict(X)
        if predictions.ndim == 1:
            predictions = predictions.reshape(-1, 1)
        return predictions

In [117]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

best_kernel = randomized_search.best_params_["svr__kernel"]
best_C = randomized_search.best_params_["svr__C"]

prediction_pipeline = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("predictor", Estimator(SVR(kernel=best_kernel, C=best_C))),
])

X_test_preprocessed = preprocessing_pipeline.fit_transform(X_test)

prediction_pipeline.fit(X, y)
predictions = prediction_pipeline.transform(X_test)

In [118]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

68099.65770537412

## Question 5

Automatically explore some preparation options using `GridSearchCV`.

In [119]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("svr", SVR(kernel=best_kernel, C=best_C)),
])

param_grid = [
    {"preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room": [False, True]}
]

grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="neg_root_mean_squared_error", verbose=3)
grid_search.fit(X, y)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=False;, score=-69430.337 total time=  15.5s
[CV 2/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=False;, score=-75573.967 total time=  15.5s
[CV 3/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=False;, score=-70312.874 total time=  17.3s
[CV 1/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=True;, score=-68822.813 total time=  17.0s
[CV 2/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=True;, score=-83178.751 total time=  13.4s
[CV 3/3] END preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room=True;, score=-70906.752 total time=  14.9s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        FeatureUnion(transformer_list=[('numerical_pipeline',
                                                                        Pipeline(steps=[('selector',
                                                                                         DataFrameSelector(attribute_names=['longitude',
                                                                                                                            'latitude',
                                                                                                                            'housing_median_age',
                                                                                                                            'total_rooms',
                                                                                                                            'total_bedrooms',
                                                                                                                            'population',
                                                                                                                            'households',
                                                                                                                            'median_income'])),
                                                                                        ('imputer',
                                                                                         SimpleImputer(strategy='median')),
                                                                                        ('attr_a...
                                                                       ('categorical_pipeline',
                                                                        Pipeline(steps=[('selector',
                                                                                         DataFrameSelector(attribute_names=['ocean_proximity'])),
                                                                                        ('label_binarizer',
                                                                                         MyLabelBinarizer())]))])),
                                       ('svr',
                                        SVR(C=63512.21010640709,
                                            kernel='linear'))]),
             param_grid=[{'preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room': [False,
                                                                                                   True]}],
             scoring='neg_root_mean_squared_error', verbose=3)

In [122]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best estimator:\t {grid_search.best_estimator_.named_steps["svr"]}")
print(f"Best score:\t {-grid_search.best_score_}")

Best parameters: {'preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room': False}
Best estimator:	 SVR(C=63512.21010640709, kernel='linear')
Best score:	 71772.39287550042


In [123]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(f"{-mean_score:.2f}\t{params}")

71772.39	{'preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room': False}
74302.77	{'preprocessing__numerical_pipeline__attr_adder__add_bedrooms_per_room': True}
